<h1 style="text-align:center">Seafood Restaurant Business Analysis with Yelp Data </h1>
<h4 style="text-align:right">Pengfei Hei, Zejin Gao, Siqi Shen, Anne Huen Wai Wong</h4>

# 1 Introduction

## 1.1 Thesis Statement

In this project, we analyze yelp data focusing on the seafood restaurant business. Our goal is to explore factors extracted from attributes and reviews that could have an influence on business ratings. And furthermore, we provide useful, analytical suggestions to seafood business owners in order to improve their Yelp ratings. Our work can be mainly divided into two parts: attributes analysis and review analysis. For the first part, we keep the attributes that over half of the businesses have and do **imputation** with **tree-based method** for these attributes. Then we fit an **multi-way ANOVA model** and find important attributes. For the second part, we do **sentiment analysis** for informative nouns in reviews. Then, we compute **correlation coefficient** between customer sentiment and business stars. We also do **chi-square test** to test the indenpendence between customers' attitudes and different words.  Finally, we combine our findings of the two parts together and give our conclusions and suggestions.

## 1.2 Data Background

This Yelp data contains about 6.69 million reviews and about 193 thousand businesses from the following cities: Montreal (Canada), Waterloo (Canada), Pittsburgh (U.S.), Charlotte (U.S.), Urbana-Champaign (U.S.), Phoenix (U.S.), Las Vegas (U.S.), Madison (U.S.), Cleveland (U.S.). There are four JSON files.

* **review json** contains 6,685,900 reviews.
* **business.json** contains information about 192,609 businesses.
* **user.json** contains information about 1,637,138 users.
* **tip.json** contains information about 1,223,094 tips written by users on businesses.

# 2 Data Filtering

talk about how to extract seafood related observations from business.json and review.json

include seafood restaurant not include steakhouse 

the length because we only want restaurant focusing on seafood and don't want other thing to interfere

the number of review > 50 because we don't want some outlier reviews

we save our new data set as ".csv".

# 3 Attribute Analysis

## 3.1 Filter Attributes
After obtaining businesses, we count all the attributes they contain and the number of businesses contained in each attribute. Some attributes only contain very few businesses and will potentially introduce bias to the analysis, thus we need to rule out attributes with enough businesses. Intuitively, we decide to drop attributes with less than 50% businesses and the table for the rest attributes shows below.

Attribute| Counts | Attribute | Counts | Attribute | Counts
------------|------|----------|--------|-----------|--------
RestaurantsTableService|225|BusinessAcceptsCreditCards|347|GoodForMeal|381
Caters|401|BikeParking|414|WiFi|426
NoiseLevel|427|RestaurantsAttire|430|Alcohol|431
HasTV|432|RestaurantsDelivery|432|RestaurantsPriceRange2|432
Ambience|432|OutdoorSeating|432|RestaurantsGoodForGroups|433
RestaurantsReservations|433|BusinessParking|433|RestaurantsTakeOut|433
GoodForKids|433

## 3.2 Imputation for Missing Values
After we filter the attributes due to its number of business, there are still many missing values. Only 5 attributes contain all businesses, and the rest all have at least 1 missing values. Thus, to better analyze the relationship between attributes and ratings, we need to impute missing values first. Since all of our attributes are categorical data, we decide to use a decision tree which is an interpretable method and easy to implement. Take attribute 'BusinessAcceptCreditCard' as an example. It has 85 missing values and its value is just 'True' or 'False', this decision tree should work well on it. We take 5 full attributes as the input, existing 'BusinessAcceptCreditCard' as output and train the model. To evaluate the performance of this model, cross-validation is applied to obtain a score. When 80% of the data is used as the training set, the score on the testing set is 0.97 which is an incredibly good result. Therefore this model could impute this attribute well. Do similar things to other attributes and we obtain a full data set.
## 3.3 Regression and Anova
Then we consider linear regression and ANOVA. We first consider a model including interaction. After fitting the model, we notice that only 'BusinessAcceptCreditCards', 'GoodForMeal', 'WiFi', 'NoiseLevel', 'RestaurantsAttire', 'Alcohol','RestaurantsGoodForGroups' and 'GoodForKids' are significant.Meanwhile, we use adjusted R squared to select variables and results are the same as significant variables. Then we fit the model and make the ANOVA table.

Attribute|Value(the first is base)|Difference|p-value
---------|-----|----------|-------
BusinessAcceptsCreditCardsTrue|False, True|0.332  |0.026
WiFi|free, no, paid| -0.181, 0.467 | 0.002
NoiseLevel|average, loud, quiet, very_loud|-0.196 ,0.146, -0.626  |0.0002
RestaurantsAttire|casual, dressy| 0.211 |0.005
Alcohol|beer_and_wine, full_bar, none|-0.220, -0.008 |0.0003
RestaurantsGoodForGroups|False, True| -0.221 |0.041
GoodForKids|False, True|-0.191  |0.017

# 4 Review Analysis

## 4.1 Data Cleaning

First, we obtain review data set with the features we need. The review data from "all_review.csv" contains 7 features. We only keep features: 'business_id', 'stars' and 'text'. Then we save the new review data set as "review_with_useful_features.csv".

Second, we do **word tokenization**, which means we need to convert text into words. This process has 7 steps: step1, convert "n't" to "not" and then connect "not" with the word after it, such as changing "wouldn't go" to "would not_go"; step2, break paragraph into words; step3, remove punctuation and abnormal string; step4, convert numbers to words; step5, convert words to lower case; step6, remove stopwords (we import "stopwords.words('english')" from python package nltk.corpus); step7, do lemmatization, such as changing "likes" to "like". 

An example is shown below.

|       business_id|  stars|                                text|                                 words
|---------------------|------|-----------------------------------------------------|-----------------------------------------------------
6xgcHeLad-VaoTIQewK84A|    3.0|  "Seems old and tired! And I wouldn't come again 3."|  \['seem','old','tired','would','not_come','three'\]


## 4.2 Sentiment Analysis

### 4.2.1 positive and negative adjectives classification

We use **Multinomial Naive Bayes Classifier** to classify adjectives in review text as positive and negative.

First, we define stars from 1 to 3 as negative and stars from 4 to 5 as positive. Then we convert stars into positive/negative tags and treat positive/negative tag as response variable. Second, we extract adjectives from tokenized words by python function "nltk.pos_tag()" (part-of-speech tagging). Third, we count the frequency of each adjective in all review texts and obtain 1200 most frequent adjectives. Fourth, we count occurrences for these 1200 adjectives in each review text and obtain the frequency matrix with the index of review as row and 1200 adjectives as column. We treat this frequency matrix as the design matrix. Fifth, we fit the multinomial Naive Bayes model with design matrix and response variable. Sixth, we do prediction with each adjective and the positive/negative prediction result is the sentiment tag for adjectives.

Here is an example: {'good': 'positive', 'delicious': 'positive', 'friendly': 'positive', 'bad': 'negative', 'decent': 'negative', 'slow': 'negative'}.

We save the **dictionary for adjectives with positive/negative tags** as "dict_adj.txt".

### 4.2.2 informative nouns in reviews

First, we extract nouns from tokenized words by python function "nltk.pos_tag()" (part-of-speech tagging). Second, we count the frequency of each noun in all review texts. Because we assume that informative nouns would show up many times in review text, we only consider nouns with frequency larger than 4000. Third, we pick up informative nouns from them manually. 

**Informative nouns** are food, lobster, crab, shrimp, oyster, fish, clam; service, waiter, waitress, chef, manager; price.

### 4.2.3 sentiment analysis for informative nouns

From this part, we do counting at the restaurant level instead of review level. That is to say, we would count some kind of word from all the review texts of each restaurant respectively.

Also, we need to obtain stars for each restaurant from "seafood_business.csv".

Then we obtain **sentiment table** for each informative nouns. First, we count the number of positive/negative adjectives in front of each informative noun. Second, we compute the proportion of positive adjectives among all the adjectives. The **sentiment table for food** is an example shown below.

business_id|positive count|negative count|positive proportion|stars
-----------|--------------|--------------|-------------------|-----
nsNONDHbV7Vudqh21uicqw|102|15|0.872|3.5
F06m2yQSPHIrb1IT7heYeQ|70|1|0.986|4.0
W7hCuNdn2gzehta6eSHzgQ|9|10|0.474|2.0

We define the proportion of positive adjectives among all the adjectives as **customer sentiment**. This variable ranges from zero to one. zero means customers are completely unsatisfied, while one means customers are completely satisfied.

## 4.3 Test and Finding

First of all, we examine three main parts that could affect the seafood business stars. They are **food**, **service** and **price**. We count the total positive/negative adjectives in front of the three words and compute the customer sentiment. To know how much the customer sentiment influences stars, we compute the **correlation coefficient** between the restaurant stars and the customer sentiment. The result is shown in the table below.

&nbsp; |  positve count | nagative count | customer sentiment | correlation
-------|----------------|----------------|--------------------|-------------
food|24047|1907|0.927|0.547
service|15458|3162|0.830|0.580
price|7869|802|0.908|0.221

We treat the first two columns of the table as a contingency table. Then we do **chi-square test** to see whether the three words and customer attitude are independent or not. The p-value is less than $2.2\times10^{-16}$, which means that customers have different attitudes towards food, service and price.

From the analysis above, we can find that food has the highest customer sentiment score, while service has the lowest. In terms of the correlation between customer sentiment and stars, customer sentiment of food and service have a strong correlation with star, while the correlation for price is not very strong. So we believe that food and service need to be further examined.

In **food category**, we find lobster, crab, shrimp, oyster, fish and clam. We do the same analysis except computing correlation. The table is shown below.

&nbsp; |  positve count | nagative count | customer sentiment
-------|----------------|----------------|-------------------
lobster|6022|374|0.942
crab|3146|479|0.868
shrimp|4379|265|0.942
oyster|802|58|0.948
fish|6646|558|0.923
clam|1154|100|0.920

The p-value for the chi-square contingency table test is less than $2.2\times10^{-16}$, which means that customers have different attitudes towards different kinds of food. Oyster has the highest customer sentiment score, while crab has the lowest.

In **service category**, we find waiter, waitress, chef and manager. We do the same analysis as the one above. The table is shown below.

&nbsp; |  positve count | nagative count | customer sentiment
-------|----------------|----------------|-------------------
waiter|6022|374|0.846
waitress|3146|479|0.834
chef|4379|265|0.945
manager|802|58|0.690

The p-value for the chi-square contingency table test is less than $2.2\times10^{-16}$, which means that customers have different attitudes towards different kinds of service. Chef has the highest customer sentiment score, while manager has the lowest.


# 5 Conclusion and Suggestion

## 5.1 conclusion

Among food, service and price, the seafood businesses need to improve service first because customers' attitudes towards service influence the business stars the most with a 0.580 correlation, while the current customers' attitudes towards service is the lowest. Among all the stuffs that provide service, only 69% of reviews about manager is associated with positive words, while above 83% of reviews about waiter/waitress/chef is associated with positive words. So the seafood businesses need to imporve the service of manager first. Besides, customers' attitudes towards food also influence the business stars a lot with a 0.547 correlation. Among all kinds of seafoods, only about 87% of reviews about crab is associated with positive words, while above 92% of reviews about other kinds of seafoods are associated with positive words. So the seafood businesses need to improve the quality of crab.


## 5.2 suggestion

1. We recommend that the seafood businesses improve the quality of service first and the quality of the seafood second.

2. We recommend that the seafood businesses increase the manager's salary to hire a person who is very good at comforting angry guests, because people have the worst evaluation towards manager.

3. We recommend that the seafood businesses offer all kinds of seafood except the crab because of the complaint about the crab, otherwise they should put more effort to improve the quality of crab.


# 6 Strength and Weakness

## 6.1 Strength

## 6.2 Weakness

# 7 Contribution

# 8 Reference

[1] https://en.wikipedia.org/wiki/Seafood_restaurant

[2] https://www.geeksforgeeks.org/python-nlp-analysis-of-restaurant-reviews/

[3] https://www.kaggle.com/iwasdata/nltk-to-classify-adjectives-as-positive-negative
